# Data Mining - Project
## Data Preparation Notebook
## Group Members
|Name|StudentID|
|:-----------:|:--------:|
|Fabio Lopes|20200597|
|Filipe Costa|20201041|
|Jorge Pereira|20201085|


In [1]:
import sys
!{sys.executable} -m pip install pyshp

You should consider upgrading via the 'C:\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import shapefile as shp

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.impute import KNNImputer
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.preprocessing import MinMaxScaler

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
df_source = pd.read_csv('./data/donors.csv')
df_source.drop('Unnamed: 0', axis = 1, inplace = True)

C:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (9,412) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
df_source.head()

In [6]:
def splitNumericAndCategorical(df):
    """Informs user on the total number of columns in the dataframe and splits it
       in 2 dataframes, one numerical and the other categorical."""
    df_num = df._get_numeric_data()
    df_cat = df.drop(df_num.columns, axis = 1)
    print(f"Total Columns: {df.shape[1]}")
    print(f"Numerical Columns: {df_num.shape[1]}")
    print(f"Categorical Columns: {df_cat.shape[1]}")
    return df_num,df_cat

In [7]:
df_numeric, df_categorical = splitNumericAndCategorical(df_source)

Total Columns: 475
Numerical Columns: 350
Categorical Columns: 125


# Drop columns which are not relevant for the analysis

In [8]:
df_aux1 = pd.DataFrame()

with open('ColumnsToKeep.txt','r') as fp:
    ColumnsToKeep = [line.strip() for line in fp.readlines()]

for column in ColumnsToKeep:
    df_aux1[column] = df_source[column]

In [9]:
df_numeric, df_categorical = splitNumericAndCategorical(df_aux1)

Total Columns: 341
Numerical Columns: 311
Categorical Columns: 30


# Prepare data

In [10]:
df_aux2 = df_aux1.copy()

## Cast dates to datetime object + create variables

In [11]:
DateColumns = ['DOB','MAXADATE','MINRDATE','MAXRDATE','LASTDATE','FISTDATE','NEXTDATE']

for column in DateColumns:
    df_aux2[column] = pd.to_datetime(df_aux1[column])

In [12]:
# Computing AGE from DOB
df_aux2['AGE'] = (pd.Timestamp('now') - df_aux2['DOB']).astype('<m8[Y]')
df_aux2.drop('DOB', axis = 1, inplace = True)

# Computing number of days elapsed from each date variable
DateColumns = ['MAXADATE','MINRDATE','MAXRDATE','LASTDATE','FISTDATE','NEXTDATE']
for column in DateColumns:
    df_aux2[f'elapsed_{column}'] = (pd.Timestamp('now') - df_aux2[column]).dt.days
    
df_aux2.drop(DateColumns, axis = 1, inplace = True)

## Create Binary Variables based on the flags

In [13]:
df_aux3 = df_aux2.copy()

In [14]:
df_aux3['PVASTATE'] = df_aux2['PVASTATE'].apply(lambda x: 1 if ( (x == 'E') or (x == 'P')) else 0)
df_aux3['NOEXCH'  ] = df_aux2['NOEXCH'  ].apply(lambda x: 0 if ( (x == 'X') ) else 1)
df_aux3['HOMEOWNR'] = df_aux2['HOMEOWNR'].apply(lambda x: 0 if ( (x == 'H') ) else 1)
df_aux3['VETERANS'] = df_aux2['VETERANS'].apply(lambda x: 0 if ( (x == 'Y') ) else 1)
df_aux3['RECINHSE'] = df_aux2['RECINHSE'].apply(lambda x: 1 if ( (x == 'X') ) else 0)
df_aux3['RECP3'   ] = df_aux2['RECP3'   ].apply(lambda x: 1 if ( (x == 'X') ) else 0)
df_aux3['RECPGVG' ] = df_aux2['RECPGVG' ].apply(lambda x: 1 if ( (x == 'X') ) else 0)
df_aux3['RECSWEEP'] = df_aux2['RECSWEEP'].apply(lambda x: 1 if ( (x == 'X') ) else 0)
df_aux3['MAJOR'   ] = df_aux2['MAJOR'   ].apply(lambda x: 1 if ( (x == 'X') ) else 0)
df_aux3['PEPSTRFL'] = df_aux2['PEPSTRFL'].apply(lambda x: 1 if ( (x == 'X') ) else 0)

## Fixes issues identified in some columns

In [15]:
df_aux4 = df_aux3.copy()

In [16]:
df_aux4['ZIP'] = df_aux3['ZIP'].str.replace('-','').astype(int)
df_aux4['GENDER'] = df_aux3['GENDER'].apply(lambda x: 1 if (x == 'F') else 2 if (x == 'M') else 0)
df_aux4['DATASRCE'] = df_aux3['DATASRCE'].apply(lambda x: 0 if (x == ' ') else int(x))
df_aux4['SOLP3'] = df_aux3['SOLP3'].apply(lambda x: 12 if (x == ' ') else int(x))
df_aux4['SOLIH'] = df_aux3['SOLIH'].apply(lambda x: 12 if (x == ' ') else int(x))
df_aux4['GEOCODE'] = df_aux3['GEOCODE'].apply(lambda x: 0 if (x == ' ') else int(x))

## Handle Domain, RFA, MDAUD Columns

In [17]:
df_aux5 = df_aux4.copy()

### Domain

In [18]:
# DOMAIN handling
# 1st byte = Urbanicity level of the donor's neighborhood
#     U=Urban=5
#     C=City=4
#     S=Suburban=3
#     T=Town=2
#     R=Rural=1

# 2nd byte = Socio-Economic status of the neighborhood
#     1 = Highest SES
#     2 = Average SES
#     3 = Lowest SES 
#     (except for Urban communities, where
#       1 = Highest SES, 
#       2 = Above average SES,
#       3 = Below average SES, 
#       4 = Lowest SES.)

def prepURBANICITY(Code):
    if Code == ' ':
        return 0 #Empty
    else:
        if Code[0] == 'R':
            return 1
        elif Code[0] == 'T':
            return 2
        elif Code[0] == 'S':
            return 3
        elif Code[0] == 'C':
            return 4
        elif Code[0] == 'U':
            return 5
        else:
            return 0
        
def prepSOCIOECON(Code):
    if Code == ' ':
        return 0 #Empty
    else:
        if (Code[0] == 'R') or (Code[0] == 'T') or (Code[0] == 'S') or Code[0] == 'C':
            if Code[1] == '1':
                return 1
            elif Code[1] == '2':
                return 3
            elif Code[1] == '3':
                return 5
            else:
                return 0
        elif Code[0] == 'U':
            if Code[1] == '1':
                return 1
            elif Code[1] == '2':
                return 2
            elif Code[1] == '3':
                return 4
            elif Code[1] == '4':
                return 5
            else:
                return 0
        else:
            return 0

df_aux5['URBANICITY'] = df_aux4['DOMAIN'].apply(prepURBANICITY)
df_aux5['SOCIOECON' ] = df_aux4['DOMAIN'].apply(prepSOCIOECON)
df_aux5.drop('DOMAIN', axis = 1, inplace = True)

### RFA

In [19]:
def prepLASTGIFTAMOUNTCATEGORY(Code):
    if Code == ' ':
        return 0 #Empty
    else:
        if Code[0] == 'A':
            return 1
        elif Code[0] == 'B':
            return 2
        elif Code[0] == 'C':
            return 3
        elif Code[0] == 'D':
            return 4
        elif Code[0] == 'E':
            return 5
        elif Code[0] == 'F':
            return 6
        elif Code[0] == 'G':
            return 7
        else:
            return 0
        
df_aux5['LASTGIFTAMOUNTCATEGORY'] = df_aux4['RFA_2A'].apply(prepLASTGIFTAMOUNTCATEGORY)
df_aux5.drop('RFA_2A', axis = 1, inplace = True)
df_aux5.drop('RFA_2R', axis = 1, inplace = True) # all donors are lapsing donors

### MDAUD

In [20]:
def prepMDONOR_GIVING_RECENCY(Code):
    if Code[0] == 'X':
        return 0 # Not a major donor
    else:
        if Code[0] == 'C':
            return 1
        elif Code[0] == 'L':
            return 2
        elif Code[0] == 'I':
            return 3
        elif Code[0] == 'D':
            return 4
        else:
            return 0
    
def prepMDONOR_GIVING_FREQUENCY(Code):
    if Code[0] == 'X':
        return 0 # Not a major donor
    else:
        if Code[0] == '1':
            return 1
        elif Code[0] == '2':
            return 2
        elif Code[0] == '5':
            return 3
        else:
            return 0
        
def prepMDONOR_GIVING_AMOUNT(Code):
    if Code[0] == 'X':
        return 0 # Not a major donor
    else:
        if Code[0] == 'L':
            return 1
        elif Code[0] == 'C':
            return 2
        elif Code[0] == 'M':
            return 3
        elif Code[0] == 'T':
            return 4
        else:
            return 0
        
df_aux5['MDONOR_GIVING_RECENCY'] = df_aux4['MDMAUD_R'].apply(prepMDONOR_GIVING_RECENCY)
df_aux5['MDONOR_GIVING_FREQUENCY'] = df_aux4['MDMAUD_F'].apply(prepMDONOR_GIVING_FREQUENCY)
df_aux5['MDONOR_GIVING_AMOUNT'] = df_aux4['MDMAUD_A'].apply(prepMDONOR_GIVING_AMOUNT)

df_aux5.drop('MDMAUD_R', axis = 1, inplace = True)
df_aux5.drop('MDMAUD_F', axis = 1, inplace = True)
df_aux5.drop('MDMAUD_A', axis = 1, inplace = True)

In [21]:
df_numeric, df_categorical = splitNumericAndCategorical(df_aux5)

Total Columns: 341
Numerical Columns: 340
Categorical Columns: 1


In [22]:
df_Prepped = df_aux5.copy()

# Scaling the data

In [23]:
df_scaling = df_Prepped.copy()

In [24]:
df_numeric, df_categorical = splitNumericAndCategorical(df_scaling)

Total Columns: 341
Numerical Columns: 340
Categorical Columns: 1


In [25]:
scaler = MinMaxScaler(feature_range=(-1, 1))
df_scaling[df_numeric.columns] = scaler.fit_transform(df_numeric)

# Reduce Dimensionality

In [26]:
# https://www.kdnuggets.com/2015/05/7-methods-data-dimensionality-reduction.html

In [27]:
df_FEreduction = df_scaling.copy()

## Drop columns based on valid observations

In [28]:
df_numeric, df_categorical = splitNumericAndCategorical(df_FEreduction)

Total Columns: 341
Numerical Columns: 340
Categorical Columns: 1


In [29]:
import math

ThresholdToIgnore = 0.4 # If more than 40% of the dataset is missing

MinimumValidSamples = math.floor(df_FEreduction.shape[0] * ThresholdToIgnore)
print(f"Minimum Valid Samples: {MinimumValidSamples}")
for column in df_FEreduction.columns:
    count = df_FEreduction[column].count()
    #print(f"{column}:{count}")
    if count < MinimumValidSamples:
        print(f"Dropped {column} , Valid Observations: {count}")
        df_FEreduction.drop(column,axis=1, inplace=True)

Minimum Valid Samples: 38164
Dropped NUMCHLD , Valid Observations: 12386


## Drop columns with high/low correlation

In [30]:
df_numeric, df_categorical = splitNumericAndCategorical(df_FEreduction)

Total Columns: 340
Numerical Columns: 339
Categorical Columns: 1


In [31]:
corPearson = df_numeric.corr().abs()

# Select upper triangle of correlation matrix
upper = corPearson.where(np.triu(np.ones(corPearson.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.90 and lower than 0.3
HighCorrelation = [column for column in upper.columns if any(upper[column] > 0.90)]
LowCorrelation = [column for column in upper.columns if max(upper[column]) < 0.30]

print(f"Dropping {len(HighCorrelation)} columns with High correlation.")
print(f"Dropping {len(LowCorrelation)} columns with Low correlation.")

# Drop features 
df_FEreduction.drop(HighCorrelation, axis=1, inplace=True)
df_FEreduction.drop(LowCorrelation, axis=1, inplace=True)

Dropping 66 columns with High correlation.
Dropping 55 columns with Low correlation.


## Remove Neighborhood data

In [32]:
with open('NeighborhoodColumns.txt','r') as fp:
    NeighborhoodColumns = [line.strip() for line in fp.readlines()]

NeighborhoodColumnsToKeep = []

columnsToDrop = list(set(NeighborhoodColumns) - set(NeighborhoodColumnsToKeep))
    
for column in columnsToDrop:
    if column in df_FEreduction.columns:
        df_FEreduction.drop(column,axis=1,inplace=True)

## Choose Clustering Variables

In [33]:
dropColumns = ['STATE','ZIP','RECSWEEP','DATASRCE','URBANICITY','SOCIOECON']

df_FEreduction.drop(dropColumns,axis=1,inplace=True)

In [34]:
df_FEreduction

,INCOME,WEALTH1,WWIIVETS,FEDGOV,WEALTH2,CARDPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,MINRAMNT,MAXRAMNT,LASTGIFT,AVGGIFT,RFA_2F,elapsed_MINRDATE,elapsed_MAXRDATE,elapsed_LASTDATE,LASTGIFTAMOUNTCATEGORY,MDONOR_GIVING_RECENCY
0,NaN,NaN,-0.636364,-0.977011,0.111111,-0.133333,-0.368421,-0.662338,-0.952069,-0.745763,-0.990,-0.997197,-0.980,-0.987071,1.000000,-0.584386,-0.718758,0.217639,-0.333333,-1.0
1,0.666667,1.000000,-0.777778,-0.977011,1.000000,-0.633333,-0.368421,-0.688312,-0.992821,-0.983051,-0.980,-0.991992,-0.950,-0.971201,-0.333333,-0.692016,-0.890172,0.217639,1.000000,-1.0
2,-0.333333,-0.777778,-0.333333,-0.977011,-0.777778,-0.166667,-0.368421,-0.662338,-0.960093,-0.779661,-0.996,-0.995596,-0.990,-0.987593,1.000000,-0.515159,-0.569926,0.217639,-0.333333,-1.0
3,-1.000000,-0.111111,-0.373737,-0.931034,-1.000000,-0.133333,-0.368421,-0.662338,-0.979730,-0.872881,-0.996,-0.997598,-0.980,-0.988932,1.000000,-0.146033,-0.788812,0.217639,-0.333333,-1.0
4,-0.333333,-0.555556,0.070707,-0.954023,NaN,0.400000,0.052632,-0.376623,-0.949113,-0.694915,-0.994,-0.995996,-0.970,-0.988827,-0.333333,-0.692016,-0.898127,0.129445,0.333333,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,NaN,NaN,-0.777778,-0.701149,NaN,-0.833333,-0.473684,-0.714286,-0.997466,-1.000000,-0.950,-0.991992,-0.950,-0.952510,-1.000000,-0.907529,-0.906082,0.041252,1.000000,-1.0
95408,1.000000,1.000000,-0.616162,-1.000000,NaN,-0.900000,-0.684211,-0.818182,-0.998522,-1.000000,-0.960,-0.993994,-0.960,-0.962523,-1.000000,-0.914856,-0.913523,-0.041252,0.333333,-1.0
95409,NaN,NaN,-0.595960,-1.000000,NaN,-0.566667,-0.263158,-0.584416,-0.990498,-0.949153,-0.994,-0.997998,-0.980,-0.985982,0.333333,-0.914856,-0.804465,-0.650071,-0.333333,-1.0
95410,1.000000,NaN,-0.595960,-0.977011,0.555556,0.166667,-0.052632,-0.220779,-0.897593,-0.661017,-0.990,-0.993594,-0.964,-0.978251,1.000000,-0.422941,-0.952784,-0.911807,0.333333,-1.0


## Missing Values

In [35]:
df_MV = df_FEreduction.copy()

In [36]:
df_numeric, df_categorical = splitNumericAndCategorical(df_MV)

Total Columns: 20
Numerical Columns: 20
Categorical Columns: 0


In [37]:
imputer = KNNImputer(n_neighbors = 5, weights = 'uniform')
df_MV[df_numeric.columns] = pd.DataFrame(imputer.fit_transform(df_numeric))

## Outlier Removal

In [38]:
df_outlier = df_MV.copy()

In [39]:
df_numeric, df_categorical = splitNumericAndCategorical(df_outlier)

Total Columns: 20
Numerical Columns: 20
Categorical Columns: 0


In [40]:
from sklearn.ensemble import IsolationForest

IsoForest = IsolationForest(contamination=0.1)

Labeling = IsoForest.fit_predict(df_numeric)

Labeling_bool = [True if x == 1 else False for x in Labeling]
Outlier_Labeling_bool = [False if x == 1 else True for x in Labeling]

df_NonOutlier = df_outlier.loc[Labeling_bool]
df_Outlier = df_outlier.loc[Outlier_Labeling_bool]

# Save Results

In [41]:
df_NonOutlier.to_csv('./Outputs/dataprep_NonOutliers.csv')
df_Outlier.to_csv('./Outputs/dataprep_Outliers.csv')
df_Prepped.to_csv('./Outputs/dataprep_final.csv')

In [42]:
print(df_NonOutlier.shape[0])
print(df_Outlier.shape[0])
print(df_Prepped.shape[0])
print(df_source.shape[0])

85870
9542
95412
95412
